# Load test data into a remote database

In [6]:
import os

DB_NAME = os.environ["CONTENT_CURATION_POSTGRES_DB_NAME"]
DB_USER = os.environ["CONTENT_CURATION_POSTGRES_USER"]
DB_PASS = os.environ["CONTENT_CURATION_POSTGRES_PASSWORD"]
DB_HOST = os.environ["CONTENT_CURATION_POSTGRES_HOST"]
DB_PORT = os.environ["CONTENT_CURATION_POSTGRES_PORT"]
DATABASE_URL = f'postgres://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

In [7]:
import psycopg2
conn = psycopg2.connect(DATABASE_URL)

In [3]:
import pandas as pd

df = pd.read_csv("test_post_data.csv", delimiter="\t")

In [4]:
cur = conn.cursor()
for idx,content in df.iterrows():
    cur.execute("""
        INSERT INTO social_post_data (internal_id, post_id, embed_html, create_utc) 
            VALUES ('%s', '%s', %s);
    """ % (content["internal_id"], content["post_id"], content["embed_html"], content["create_utc"]))
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "social_post_data_pkey"
DETAIL:  Key (post_id)=(https://www.reddit.com/r/AskReddit/comments/1e7vx71/whats_the_worst_movie_you_watched_in_your_entire/) already exists.


In [8]:
cur = conn.cursor()

cur.execute("""
    SELECT MAX(internal_id)
    FROM social_post_data
""")
max_id = cur.fetchone()
max_id

(None,)

In [5]:
cur.close()
conn.close()